<a href="https://colab.research.google.com/github/Ayush-mishra-0-0/NLP/blob/main/interiit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4 pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Input,
    MultiHeadAttention, LayerNormalization
)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import nltk
import transformers
from textblob import TextBlob
import yfinance as yf

In [ ]:
folder= r"/content/drive/MyDrive/data"

df= pd.read_csv(os.path.join(folder, "BTC_2019_2023_1d.csv"))
df.head()

,Unnamed: 0,datetime,open,high,low,close,volume
0,0,2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291
1,1,2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373
2,2,2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955
3,3,2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922
4,4,2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_crypto_news(url="https://finance.yahoo.com/topic/crypto/"):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data. HTTP Status code: {response.status_code}")
            return None
        soup = BeautifulSoup(response.content, 'html.parser')
        articles = soup.find_all('a', class_='subtle-link')
        data = []
        for article in articles:
            headline = article.text.strip()
            link = article['href'] if article.has_attr('href') else None
            full_link = f"https://finance.yahoo.com{link}" if link and not link.startswith("http") else link

            if headline:
                data.append({
                    'headline': headline,
                    'link': full_link
                })
        return pd.DataFrame(data)

    except Exception as e:
        print(f"Error fetching news: {e}")
        return None

crypto_news = fetch_crypto_news()

In [ ]:
len(crypto_news)

85

In [ ]:
crypto_news['link'][7]

'https://finance.yahoo.com/news/spot-bitcoin-etf-options-hit-215139627.html'

In [ ]:
def fetch_article_content(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch article. URL: {url}, HTTP Status: {response.status_code}")
            return None
        soup = BeautifulSoup(response.content, 'html.parser')
        content_div = soup.find('div', class_='body')
        if not content_div:
            print(f"Could not find content for URL: {url}")
            return None
        paragraphs = content_div.find_all('p')
        article_content = ' '.join([p.text.strip() for p in paragraphs if p.text])

        return article_content

    except Exception as e:
        print(f"Error fetching article content from {url}: {e}")
        return None

def scrape_all_articles(news_df):
    news_df['content'] = news_df['link'].apply(fetch_article_content)
    return news_df


crypto_news = fetch_crypto_news()
if crypto_news is not None:
    updated_news = scrape_all_articles(crypto_news)
    print(updated_news.head())

    updated_news.to_csv('crypto_news_with_content.csv', index=False)

Failed to fetch article. URL: https://finance.yahoo.com/news/trump-said-eye-crypto-lawyer-230734873.html, HTTP Status: 404
Failed to fetch article. URL: https://finance.yahoo.com/news/mstu-bitcoin-leverage-spikes-900-222440549.html, HTTP Status: 404
Failed to fetch article. URL: https://finance.yahoo.com/news/spot-bitcoin-etf-options-hit-215139627.html, HTTP Status: 404
Failed to fetch article. URL: https://finance.yahoo.com/news/crypto-conference-circuit-good-crypto-203156035.html, HTTP Status: 404
Could not find content for URL: https://www.yahoo.com/tech/bitcoin-close-breaking-94-000-130200413.html
Failed to fetch article. URL: https://finance.yahoo.com/news/defi-renaissance-161632948.html, HTTP Status: 404
Failed to fetch article. URL: https://finance.yahoo.com/news/coinbase-delists-wrapped-bitcoin-wbtc-054039766.html, HTTP Status: 404
Could not find content for URL: https://finance.yahoo.com/personal-finance/mortgages/
Could not find content for URL: https://finance.yahoo.com/pers

In [ ]:
data = pd.read_csv(r"/content/crypto_news_with_content.csv")

In [ ]:
# remove nan content
data = data.dropna(subset=['content'])

In [ ]:
data.head()

,headline,link,content
0,"Bitcoin Climbs Closer to $100,000 on Trump’s S...",https://finance.yahoo.com/news/bitcoin-hits-re...,"(Bloomberg) -- Bitcoin hit $95,000 for the fir..."
1,"Bitcoin Crosses $97K, Continuing Wild ‘Trump T...",https://finance.yahoo.com/news/bitcoin-crosses...,Bitcoin {{BTC}} momentum shows no signs of slo...
2,"Bitcoin breaks $96,000 for first time on optim...",https://finance.yahoo.com/news/bitcoin-breaks-...,SINGAPORE (Reuters) - Bitcoin's march toward $...
3,Michael Saylor predicts bitcoin will continue ...,https://finance.yahoo.com/news/michael-saylor-...,"Michael Saylor, the founder of MicroStrategy, ..."
6,Trump eyes official crypto lead or czar Presid...,https://finance.yahoo.com/news/trump-eyes-offi...,"In what would be the first role of its kind, P..."


In [ ]:
data.shape

(52, 3)

In [ ]:
class CryptoTradingFramework:
    def __init__(self, data_dir, timeframes=['1d', '1h', '4h']):
        """
        Initialize the trading framework with multi-timeframe data

        Parameters:
        - data_dir: Directory containing cryptocurrency data files
        - timeframes: List of timeframes to analyze
        """
        self.data_dir = data_dir
        self.timeframes = timeframes
        self.models = {}
        self.scalers = {}

    def load_data(self):
        """
        Load cryptocurrency data from multiple timeframes
        """
        self.datasets = {}
        for timeframe in self.timeframes:
            filename = f'BTC_2019_2023_{timeframe}.csv'
            file_path = os.path.join(self.data_dir, filename)

            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                # Ensure consistent column names
                df.columns = [col.lower() for col in df.columns]

                # Basic preprocessing
                df['datetime'] = pd.to_datetime(df['datetime'])
                df.set_index('datetime', inplace=True)

                # Sort and clean data
                df.sort_index(inplace=True)
                df.dropna(inplace=True)

                self.datasets[timeframe] = df

        return self.datasets

    def calculate_technical_indicators(self, df):
        """
        Calculate comprehensive technical indicators

        Parameters:
        - df: Input DataFrame

        Returns:
        - DataFrame with additional technical indicators
        """
        # Moving Averages
        df['sma_20'] = df['close'].rolling(window=20).mean()
        df['sma_50'] = df['close'].rolling(window=50).mean()
        df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()

        # Bollinger Bands
        df['bb_middle'] = df['close'].rolling(window=20).mean()
        df['bb_std'] = df['close'].rolling(window=20).std()
        df['bb_upper'] = df['bb_middle'] + 2 * df['bb_std']
        df['bb_lower'] = df['bb_middle'] - 2 * df['bb_std']

        # Relative Strength Index (RSI)
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))

        # MACD
        exp1 = df['close'].ewm(span=12, adjust=False).mean()
        exp2 = df['close'].ewm(span=26, adjust=False).mean()
        df['macd'] = exp1 - exp2
        df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()

        # Volatility Indicators
        df['atr'] = df['high'] - df['low']  # Average True Range
        df['volatility'] = df['atr'].rolling(window=14).mean()

        return df

    def prepare_ml_features(self, df, lookback=30):
        """
        Prepare features for machine learning models

        Parameters:
        - df: Input DataFrame
        - lookback: Number of historical periods to consider

        Returns:
        - Features and target variable
        """
        # Calculate technical indicators
        df = self.calculate_technical_indicators(df)

        # Prepare features
        feature_columns = [
            'sma_20', 'sma_50', 'ema_20',
            'bb_upper', 'bb_lower',
            'rsi', 'macd', 'signal_line',
            'volatility'
        ]

        # Create lagged features
        for col in ['close', 'volume'] + feature_columns:
            for lag in range(1, lookback + 1):
                # df[f'{col}_lag_{lag}'] = df[col].shift(lag)
                lags = {f'{col}_lag_{lag}': df[col].shift(lag) for lag in range(1, lookback + 1)}
                df = pd.concat([df, pd.DataFrame(lags)], axis=1)


        # Calculate returns
        df['returns'] = df['close'].pct_change()

        # Drop NaN values
        df.dropna(inplace=True)

        # Prepare X and y
        features = df[feature_columns]
        target = df['returns']

        return features, target

    def build_lstm_model(self, input_shape):
        """
        Build an advanced LSTM model with attention mechanism

        Parameters:
        - input_shape: Shape of input data

        Returns:
        - Compiled Keras model
        """
        inputs = Input(shape=input_shape)

        # LSTM layers with attention
        x = LSTM(64, return_sequences=True)(inputs)
        x = Dropout(0.2)(x)

        # Multi-head attention
        attention = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
        x = LayerNormalization()(x + attention)

        x = LSTM(32)(x)
        x = Dropout(0.2)(x)

        outputs = Dense(1, activation='linear')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')

        return model

    def build_ensemble_model(self, features, target):
        """
        Build an ensemble of machine learning models

        Parameters:
        - features: Input features
        - target: Target variable

        Returns:
        - Dictionary of trained models
        """
        # Random Forest
        rf_model = RandomForestRegressor(
            n_estimators=100,
            random_state=42
        )
        rf_model.fit(features, target)

        # Gradient Boosting
        gb_model = GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3
        )
        gb_model.fit(features, target)

        return {
            'random_forest': rf_model,
            'gradient_boosting': gb_model
        }

    def fetch_sentiment_data(self):
        """
        Fetch sentiment data from various sources

        Returns:
        - Sentiment scores for Bitcoin
        """
        # Fetch Bitcoin-related news and social media data
        try:
            btc_data = yf.Ticker("BTC-USD")
            news = btc_data.news

            # Simple sentiment analysis
            sentiments = []
            for article in news[:10]:  # Limit to 10 recent articles
                blob = TextBlob(article['title'] + " " + article['description'])
                sentiments.append(blob.sentiment.polarity)

            return np.mean(sentiments)
        except Exception as e:
            print(f"Sentiment data fetch error: {e}")
            return 0

    def generate_trading_signals(self, predictions, sentiment_score):
        """
        Generate trading signals combining ML predictions and sentiment

        Parameters:
        - predictions: Model predictions
        - sentiment_score: Market sentiment score

        Returns:
        - Trading signals (-1: sell, 0: hold, 1: buy)
        """
        # Combine multiple signals
        signals = np.zeros_like(predictions)

        # ML prediction signals
        signals[predictions > predictions.mean() + 0.5 * predictions.std()] = 1
        signals[predictions < predictions.mean() - 0.5 * predictions.std()] = -1

        # Incorporate sentiment
        if sentiment_score > 0.1:
            signals += 0.5
        elif sentiment_score < -0.1:
            signals -= 0.5

        return np.clip(signals, -1, 1)

    def backtest_strategy(self, df, signals, transaction_cost=0.001):
        """
        Backtest the trading strategy

        Parameters:
        - df: Price data
        - signals: Trading signals
        - transaction_cost: Cost per trade

        Returns:
        - Performance metrics
        """
        # Calculate returns
        df['strategy_return'] = signals * df['returns'].shift(1)
        df['strategy_return'] -= transaction_cost * np.abs(signals - signals.shift(1))

        # Cumulative performance
        df['cumulative_strategy_return'] = (1 + df['strategy_return']).cumprod()

        # Performance metrics
        sharpe_ratio = df['strategy_return'].mean() / df['strategy_return'].std() * np.sqrt(252)
        max_drawdown = (df['cumulative_strategy_return'] / df['cumulative_strategy_return'].cummax() - 1).min()

        return {
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'total_return': df['cumulative_strategy_return'].iloc[-1] - 1
        }

    def run_comprehensive_strategy(self):
        """
        Run the comprehensive trading strategy
        """
        # Load multi-timeframe data
        datasets = self.load_data()

        results = {}
        for timeframe, df in datasets.items():
            # Prepare features
            features, target = self.prepare_ml_features(df)

            # Build and train models
            self.models[timeframe] = {
                'ensemble': self.build_ensemble_model(features, target)
            }

            # LSTM model
            X = features.values.reshape(
                (features.shape[0], 1, features.shape[1])
            )
            lstm_model = self.build_lstm_model(X.shape[1:])
            lstm_model.fit(X, target, epochs=50, batch_size=32, verbose=0)
            self.models[timeframe]['lstm'] = lstm_model

            # Make predictions
            predictions = {}
            for model_name, model in self.models[timeframe]['ensemble'].items():
                predictions[model_name] = model.predict(features)

            # LSTM predictions
            lstm_preds = self.models[timeframe]['lstm'].predict(X)
            predictions['lstm'] = lstm_preds

            # Fetch sentiment
            sentiment_score = self.fetch_sentiment_data()

            combined_predictions = np.mean(
                [np.array(pred).ravel() for pred in predictions.values()],
                axis=0
            )

            signals = self.generate_trading_signals(combined_predictions, sentiment_score)

            # Backtest
            results[timeframe] = self.backtest_strategy(df, signals)

        return results

In [ ]:
# Example usage
framework = CryptoTradingFramework(folder)
strategy_results = framework.run_comprehensive_strategy()

In [ ]:
pip install pandas numpy scikit-learn joblib ta untrade